In [ ]:
import pandas as pd
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [ ]:
file_url = '../Dataset/breast-cancer-wisconsin.data'

In [ ]:
col_names = ['Sample code number','Clump Thickness','Uniformity of Cell Size','Uniformity of Cell Shape','Marginal Adhesion','Single Epithelial Cell Size',
'Bare Nuclei','Bland Chromatin','Normal Nucleoli','Mitoses','Class']

In [ ]:
df = pd.read_csv(file_url, header=None, names=col_names, na_values='?')

In [ ]:
df.head()

In [ ]:
df.fillna(0, inplace=True)

In [ ]:
y = df.pop('Class')

In [ ]:
X = df.drop('Sample code number', axis=1)

In [ ]:
X.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=888)

In [ ]:
rf_model = RandomForestClassifier(random_state=1)

In [ ]:
rf_model.fit(X_train, y_train)

In [ ]:
rf_model.predict([X_test.iloc[0,]])

In [ ]:
joblib.dump(rf_model, "model.pkl") 

In [ ]:
import socket
import threading
import requests
import json
from flask import Flask, jsonify, request
import numpy as np

In [ ]:
ip_address = socket.gethostbyname(socket.gethostname()) + '8080'
ip_address

In [ ]:
app = Flask(__name__)

In [ ]:
trained_model = joblib.load("model.pkl")

In [ ]:
@app.route('/api', methods=['POST'])
def predict():
  data = request.get_json()
  prediction = trained_model.predict(data)
  str_pred = np.array2string(prediction)
  return jsonify(str_pred)

In [ ]:
flask_thread = threading.Thread(target=app.run, kwargs={'host':'0.0.0.0','port':8080})
flask_thread.start()

In [ ]:
record = X_test.iloc[0,].to_list()
record

In [ ]:
j_data = json.dumps([record])

In [ ]:
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

In [ ]:
r = requests.post(f"http://{ip_address}/api", data=j_data, headers=headers)
r.text